In [2]:
import os
import pprint
import json
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
#os.getenv('OPENAI_API_KEY')

In [5]:
from openai import OpenAI

In [6]:
client = OpenAI()

## Function Calling
connect large language models to external tools

### Trying to get real-time data using Chat API Completion

In [16]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": "What is the weather like today in London?",
        }
    ]
)
print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am unable to provide real-time weather updates. I recommend checking a reliable weather website or app for the most current information on the weather in London.', role='assistant', function_call=None, tool_calls=None))


### Get real-time data using function calling

In [14]:
functions = [
    {
      "type": "function",
      "function": {
        "name": "get_current_weather",
        "description": "Get the current weather for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city, e.g., San Francisco, London"
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "The temperature unit to use. use the correct unit here based on the user's location."
            }
          },
          "required": ["location", "unit"]
        }
      }
    }]

### Chat model auto decides whether to use a function call or get and return output response text

In [28]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    tools=functions,
    tool_choice="auto",  # auto is default
    messages=[
        {
            "role": "user",
            "content": "Who won the Euro Cup 2024?",
        }
    ]
)
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The Euro Cup 2024 has not yet occurred. The tournament is scheduled for June and July of 2024 in Germany. As of now, the winner has not been determined.', role='assistant', function_call=None, tool_calls=None))

##### No text response from chat model but intead returning to call a function



In [20]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    tools=functions,
    tool_choice="required",  # auto is default
    messages=[
        {
            "role": "user",
            "content": "Who won the last FIFA football?",
        }
    ]
)
response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_clSbR9HXQi3b3H8gGdELHyrQ', function=Function(arguments='{"location":"Paris","unit":"Celsius"}', name='get_current_weather'), type='function')])

#### ^ Based on function, model auto detects the correct unit to be used.
London: Celsius, New York: Fahrenheit

In [126]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    tools=tools,
    tool_choice="auto",  # auto is default
    messages=[
        {
            "role": "user",
            "content": "Which team the most FIFA World Cup wins",
        }
    ]
)
response.choices[0].message

ChatCompletionMessage(content='The team with the most FIFA World Cup wins is Brazil, with a total of 5 titles.', role='assistant', function_call=None, tool_calls=None)

##### Model auto decided to not suggest a function call as it has the answer for query

### Force model to use function call instead of output response

In [127]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    tools=tools,
    tool_choice="required",
    messages=[
        {
            "role": "user",
            "content": "Which team the most FIFA World Cup wins",
        }
    ]
)
response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_UQomCCM2FoQnImM0qgJ5iQ9x', function=Function(arguments='{"location":"Moscow","unit":"Celsius"}', name='get_current_weather'), type='function')])

##### Even the model had a response for the query, but still suggested to call function as it was forced by the chat arguments

##### Example dummy function to return the weather. This could be your backend API or an external API

In [23]:
def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "new york" in location.lower():
        return json.dumps({"location": "New York", "temperature": "72", "unit": unit})
    elif "london" in location.lower():
        return json.dumps({"location": "London", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

### All in a single place to test function calling

In [25]:
messages = [
        {
            "role": "user",
            "content": "What is the weather like today in London?",
        }
    ]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    tools=functions,
    tool_choice="auto",
    messages=messages
)

print(response.choices[0].message, '\n\n')

tool_calls = response.choices[0].message.tool_calls

# check if the model wanted to call a function

if tool_calls:
    # the JSON response may not always be valid; be sure to handle errors
    
    available_functions = {
        "get_current_weather": get_current_weather,
    }
    
    # only one function in this example, but you can have multiple
    
    messages.append(response.choices[0].message)  # extend conversation with assistant's reply
    
    # send the info for each function call and function response to the model
    tool_call = tool_calls[0]
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )
    # extend conversation with function response

    print(messages, '\n\n')
    
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    
    # get a new response from the model where it can see the function response
    
    print(second_response.choices[0].message.content)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_poWae8jbqMg0kXeGO7HFgMD7', function=Function(arguments='{"location":"London","unit":"Celsius"}', name='get_current_weather'), type='function')]) 


[{'role': 'user', 'content': 'What is the weather like today in London?'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_poWae8jbqMg0kXeGO7HFgMD7', function=Function(arguments='{"location":"London","unit":"Celsius"}', name='get_current_weather'), type='function')]), {'tool_call_id': 'call_poWae8jbqMg0kXeGO7HFgMD7', 'role': 'tool', 'name': 'get_current_weather', 'content': '{"location": "London", "temperature": "22", "unit": "Celsius"}'}] 


The weather in London today is 22 degrees Celsius.


### Make function calling re-usable by moving to a generic function

In [133]:
def get_live_weather_update(messages, response, function):
    print("Initial Response: ", '\n\n')
    pprint.pp(response.choices[0].message, indent=4)
    print('\n\n')
    
    tool_calls = response.choices[0].message.tool_calls

    if not tool_calls:
        return response.choices[0].message.content

    available_functions = {
        function.__name__: function,
    }

    # extend conversation with assistant's reply
    messages.append(response.choices[0].message)
    
    # send the info for each function call and function response to the model
    tool_call = tool_calls[0]
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )

    print("Response from our weather API: ", '\n\n')
    pprint.pp(function_response, indent=4)
    print('\n\n')

    # appending new message to existing messages for sending back to chat model
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

    print("Messages going to model: ", '\n\n')
    pprint.pp(messages, indent=4, depth=2)
    print('\n\n')
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    
    print("Final response from model: ", '\n\n')
    pprint.pp(response.choices[0].message, indent=4, depth=2)

    # re-send messages to chat model to generate output based on actual function call
    return response.choices[0].message.content

In [134]:
messages = [
        {
            "role": "user",
            "content": "What was the weather like today in London?",
        }
    ]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    tools=tools,
    tool_choice="auto",
    messages=messages
)

get_live_weather_update(messages, response, get_current_weather)

Initial Response:  


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_eQWT1va21pGjP8d2BkVgQd0C', function=Function(arguments='{"location":"London","unit":"Celsius"}', name='get_current_weather'), type='function')])



Response from our weather API:  


'{"location": "London", "temperature": "22", "unit": "Celsius"}'



Messages going to model:  


[   {'role': 'user', 'content': 'What was the weather like today in London?'},
    ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_eQWT1va21pGjP8d2BkVgQd0C', function=Function(arguments='{"location":"London","unit":"Celsius"}', name='get_current_weather'), type='function')]),
    {   'tool_call_id': 'call_eQWT1va21pGjP8d2BkVgQd0C',
        'role': 'tool',
        'name': 'get_current_weather',
        'content': '{"location": "London", "temperature": "22", "unit": '
                   '"Cels

'The weather in London today is 22 degrees Celsius.'

In [136]:
messages = [
        {
            "role": "user",
            "content": "What was the weather like today in New York?",
        }
    ]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    tools=tools,
    tool_choice="auto",
    messages=messages
)

get_live_weather_update(messages, response, get_current_weather)

Initial Response:  


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jfa55xHeoirpLVOYJKS0qkxI', function=Function(arguments='{"location":"New York","unit":"Celsius"}', name='get_current_weather'), type='function')])



Response from our weather API:  


'{"location": "New York", "temperature": "72", "unit": "Celsius"}'



Messages going to model:  


[   {'role': 'user', 'content': 'What was the weather like today in New York?'},
    ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jfa55xHeoirpLVOYJKS0qkxI', function=Function(arguments='{"location":"New York","unit":"Celsius"}', name='get_current_weather'), type='function')]),
    {   'tool_call_id': 'call_jfa55xHeoirpLVOYJKS0qkxI',
        'role': 'tool',
        'name': 'get_current_weather',
        'content': '{"location": "New York", "temperature": "72", "unit": '
               

'The weather in New York today was 72 degrees Celsius.'

In [137]:
messages = [
        {
            "role": "user",
            "content": "Who was the first president of US?",
        }
    ]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    tools=tools,
    tool_choice="auto",
    messages=messages
)

get_live_weather_update(messages, response, get_current_weather)

Initial Response:  


ChatCompletionMessage(content='The first president of the United States was George Washington.', role='assistant', function_call=None, tool_calls=None)





'The first president of the United States was George Washington.'